In [1]:
import statsmodels.formula.api as smf
import pandas as pd
import os 
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family']='Arial'

- `extend`
    - `avg`~...
        - w/ conveyor
        - w/o conveyor
    - `attrition`~...
        - w/ conveyor
        - w/o conveyor
- `charge`
    - `avg`~...
        - w/ conveyor
        - w/o conveyor
    - `attrition`~...
        - w/ conveyor
        - w/o conveyor

In [76]:
# funcs def.
def showSig(x):
    x=round(x,4)
    if x<0.01:
        return(f'*** {x}')
    if x<0.05:
        return(f'** {x}')
    if x<0.1:
        return(f'* {x}')
    else:
        return(x)
    
def modelSumm(model, verbose=False, sig=False):
    if verbose==True:
        return model.summary()
    if sig==True:
        return pd.DataFrame({ 'p_value': model.pvalues.apply(showSig), 'std_error': model.bse.apply(lambda x: round(x,4)) })
    return pd.DataFrame({ 'p_value': model.pvalues.apply(lambda x: round(x,4)), 'std_error': model.bse.apply(lambda x: round(x,4))})

In [83]:
extend=pd.read_csv('wages_extend_month.csv')
charge=pd.read_csv('wages_charge_month.csv')
extend['pschange']=extend.loc[extend['pschange']<140, 'pschange'].replace(np.inf, 1000)
ind_vars=["avgprice","cus1price","mbpricing","mbpay","numbuyers","pschange","workers","allps","conveyor"]

In [84]:
infRec=[]
for i,r in extend.iterrows():
    if np.isinf(r['pschange']):
        infRec.append(i)

extend.drop(infRec, inplace=True)

In [87]:
ext_avg_wc=smf.ols(f'avg_pay ~ avgprice + cus1price + mbpricing + mbpay + numbuyers + pschange + workers + allps + conveyor',
            data=extend).fit()

ext_avg_woc=smf.ols(f'avg_pay ~ avgprice + cus1price + mbpricing + mbpay + numbuyers + pschange + workers + allps',
            data=extend).fit()

modelSumm(ext_avg_wc, sig=1)

,p_value,std_error
Intercept,*** 0.0038,2369.5219
conveyor[T.A-2],0.1141,251.7390
conveyor[T.A-3],0.5406,270.8121
conveyor[T.A-4],* 0.051,197.4554
conveyor[T.B-1],0.4862,205.7432
conveyor[T.B-2],0.9213,462.6585
conveyor[T.B-3],0.2143,269.2787
conveyor[T.B-4],0.2019,265.0993
conveyor[T.C-1],** 0.0443,418.2839
conveyor[T.C-2],** 0.035,396.9811


In [88]:
modelSumm(ext_avg_woc, sig=1)

,p_value,std_error
Intercept,*** 0.0,2433.5004
avgprice,0.3933,62.8491
cus1price,0.3092,67.8037
mbpricing,0.3276,438.3599
mbpay,* 0.0915,321.4894
numbuyers,0.6309,34.9384
pschange,0.9133,1.3479
workers,0.7889,4.1690
allps,*** 0.0006,0.0065
